# Feature Engineering

- Reduce Dimensional Model:
  - Codify columns
  - Reduce Categories
  - Group variables
  - Drop duplicates

In [1]:
# Libraries
import pandas as pd
import numpy as np
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read Data
df = pd.read_parquet('/content/drive/MyDrive/No Country Proyecto/Data/Clean/Icfes2023.parquet')
df.head(1)

,ESTU_TIPODOCUMENTO,ESTU_GENERO,PERIODO,ESTU_DEPTO_RESIDE,ESTU_MCPIO_RESIDE,ESTU_PRESENTACIONSABADO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,...,PERCENTIL_SOCIALES_CIUDADANAS,DESEMP_SOCIALES_CIUDADANAS,PUNT_INGLES,PERCENTIL_INGLES,DESEMP_INGLES,PUNT_GLOBAL,PERCENTIL_GLOBAL,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO
0,TI,F,1,CUNDINAMARCA,CHÍA,No,3,5 a 6,4.0,Educación profesional completa,...,97,4,87.0,94.0,B+,422,100.0,67.532814,4.0,4.0


In [4]:
df.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'FAMI_NUMLIBROS', 'FAMI_COMELECHEDERIVADOS',
       'FAMI_COMECARNEPESCADOHUEVO', 'FAMI_COMECEREALFRUTOSLEGUMBRE',
       'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA', 'COLE_NOMBRE_ESTABLECIMIENTO',
       'COLE_NATURALEZA', 'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION', 'COLE_JORNADA',
       'COLE_MCPIO_UBICACION', 'ESTU_MCPIO_PRESENTACION',
       'PUNT_LECTURA_CRITICA', 'PERCENTIL_LECTURA_CRITICA',
       'DESEMP_LECTURA_CRITICA', 'PUNT_MATEMATICAS', 'PERCENTIL_MATEMATICAS',
       'DESEMP_MATEMATICAS', 'PUNT_C_NATURALES', 'PERCENTIL_C_NATURALES',
       'DESEMP_C_N

## Functions

In [5]:
def values_per_attribute(dataframe:pd.DataFrame, attribute:str):
  len_dataset = len(dataframe)
  categorias_total = 0
  for value in dataframe[attribute.upper()].unique():
    categorias_total += 1
    if str(value) == 'nan':
      len_attribute = len(dataframe.loc[dataframe[attribute.upper()].isna()])
    else:
      len_attribute = len(dataframe.loc[dataframe[attribute.upper()]==value])
    print(f'- {value}: {len_attribute}, porcentaje: {round(len_attribute*100/len_dataset,3)}%')
  print('total de categorias:',categorias_total)

def eliminate_values_per_attribute_limited(dataframe:pd.DataFrame, attribute:str, limit: float):
  df_temp = dataframe.copy()
  len_dataset = len(df_temp)
  attributes_to_eliminate = []
  for value in df_temp[attribute.upper()].unique():
    if str(value) == 'nan':
      len_attribute = len(df_temp.loc[df_temp[attribute.upper()].isna()])
    else:
      len_attribute = len(df_temp.loc[df_temp[attribute.upper()]==value])
    if round(len_attribute*100/len_dataset,3)<limit:
      attributes_to_eliminate.append(value)
  return df_temp.loc[~df_temp[attribute].isin(attributes_to_eliminate)]

def values_per_attribute_limited(dataframe:pd.DataFrame, attribute:str, limit: float):
  len_dataset = len(dataframe)
  porcentaje_total = 0
  categorias_total = 0
  for value in dataframe[attribute.upper()].unique():
    if str(value) == 'nan':
      len_attribute = len(dataframe.loc[dataframe[attribute.upper()].isna()])
    else:
      len_attribute = len(dataframe.loc[dataframe[attribute.upper()]==value])
    if round(len_attribute*100/len_dataset,3)<limit:
      porcentaje_total += round(len_attribute*100/len_dataset,3)
      categorias_total += 1
      print(f'- {value}: {len_attribute}, porcentaje: {round(len_attribute*100/len_dataset,3)}%')
  print('porcentaje',porcentaje_total,'%')
  print('total categorias',categorias_total)

def to_quartil(percentil,q1,q2,q3):
    if percentil <= q1:
        return 'Q1'
    elif percentil <= q2:
        return 'Q2'
    elif percentil <= q3:
        return 'Q3'
    else:
        return 'Q4'

def create_quartil_cols(dataframe:pd.DataFrame):
  percentil_columns = [name for name in dataframe.columns if 'PERCENTIL' in name]
  percentil_columns.remove('PERCENTIL_GLOBAL')
  quartil_columns = [name.replace('PERCENTIL','CUARTIL') for name in percentil_columns]
  percentil_results = {}
  for idx,percentil_col in enumerate(percentil_columns):
    q1 = dataframe[percentil_col].quantile(0.25)
    q2 = dataframe[percentil_col].quantile(0.50)
    q3 = dataframe[percentil_col].quantile(0.75)
    df[quartil_columns[idx]] = df[percentil_col].apply(lambda x: to_quartil(x,q1,q2,q3))

## Eliminate granularity

- Eliminate the next columns: PUNT_LECTURA_CRITICA,PUNT_MATEMATICAS,PUNT_C_NATURALES,PUNT_SOCIALES_CIUDADANAS,PUNT_INGLES

In [6]:
df.drop(columns=['PUNT_LECTURA_CRITICA','PUNT_MATEMATICAS','PUNT_C_NATURALES','PUNT_SOCIALES_CIUDADANAS','PUNT_INGLES'],inplace=True)

In [7]:
# Document type
attributes_NOTTO_analyse = ['ESTU_DEPTO_RESIDE','ESTU_MCPIO_RESIDE','COLE_NOMBRE_ESTABLECIMIENTO','COLE_NOMBRE_SEDE','COLE_MCPIO_UBICACION',
                            'COLE_MCPIO_PRESENTACION','PERCENTIL_LECTURA_CRITICA','DESEMP_LECTURA_CRITICA','PERCENTIL_MATEMATICAS','DESEMP_MATEMATICAS',
                            'PERCENTIL_C_NATURALES','DESEMP_C_NATURALES','PERCENTIL_SOCIALES_CIUDADANAS','DESEMP_SOCIALES_CIUDADANAS','PUNT_GLOBAL',
                            'PERCENTIL_INGLES', 'DESEMP_INGLES','PERCENTIL_GLOBAL','ESTU_INSE_INDIVIDUAL','PERCENTIL_ESPECIAL_GLOBAL','ESTU_MCPIO_PRESENTACION']
for attribute in df.columns:
  if attribute not in attributes_NOTTO_analyse:
    print(attribute.capitalize())
    values_per_attribute(df, attribute)

Estu_tipodocumento
- TI: 358212, porcentaje: 88.258%
- CC: 47657, porcentaje: 11.742%
total de categorias: 2
Estu_genero
- F: 220145, porcentaje: 54.24%
- M: 185724, porcentaje: 45.76%
total de categorias: 2
Periodo
- 1: 10202, porcentaje: 2.514%
- 2: 395667, porcentaje: 97.486%
total de categorias: 2
Estu_presentacionsabado
- No: 369501, porcentaje: 91.039%
- Si: 36368, porcentaje: 8.961%
total de categorias: 2
Fami_estratovivienda
- 3: 100831, porcentaje: 24.843%
- 2: 151454, porcentaje: 37.316%
- 1: 110981, porcentaje: 27.344%
- 5: 9521, porcentaje: 2.346%
- 4: 27890, porcentaje: 6.872%
- 6: 5192, porcentaje: 1.279%
total de categorias: 6
Fami_personashogar
- 5 a 6: 113936, porcentaje: 28.072%
- 3 a 4: 218908, porcentaje: 53.936%
- 1 a 2: 39951, porcentaje: 9.843%
- 7 a 8: 24828, porcentaje: 6.117%
- 9 o más: 8246, porcentaje: 2.032%
total de categorias: 5
Fami_cuartoshogar
- 4.0: 49400, porcentaje: 12.171%
- 2.0: 154509, porcentaje: 38.069%
- 3.0: 162689, porcentaje: 40.084%
- 6.0:

In [8]:
create_quartil_cols(df)
df.drop(columns=['PERCENTIL_LECTURA_CRITICA','PERCENTIL_MATEMATICAS','PERCENTIL_C_NATURALES',
                 'PERCENTIL_SOCIALES_CIUDADANAS','PERCENTIL_INGLES'],inplace=True)

In [9]:
df.head(1)

,ESTU_TIPODOCUMENTO,ESTU_GENERO,PERIODO,ESTU_DEPTO_RESIDE,ESTU_MCPIO_RESIDE,ESTU_PRESENTACIONSABADO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,...,PUNT_GLOBAL,PERCENTIL_GLOBAL,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,CUARTIL_LECTURA_CRITICA,CUARTIL_MATEMATICAS,CUARTIL_C_NATURALES,CUARTIL_SOCIALES_CIUDADANAS,CUARTIL_INGLES
0,TI,F,1,CUNDINAMARCA,CHÍA,No,3,5 a 6,4.0,Educación profesional completa,...,422,100.0,67.532814,4.0,4.0,Q4,Q4,Q4,Q4,Q4


In [10]:
df.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'FAMI_NUMLIBROS', 'FAMI_COMELECHEDERIVADOS',
       'FAMI_COMECARNEPESCADOHUEVO', 'FAMI_COMECEREALFRUTOSLEGUMBRE',
       'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA', 'COLE_NOMBRE_ESTABLECIMIENTO',
       'COLE_NATURALEZA', 'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION', 'COLE_JORNADA',
       'COLE_MCPIO_UBICACION', 'ESTU_MCPIO_PRESENTACION',
       'DESEMP_LECTURA_CRITICA', 'DESEMP_MATEMATICAS', 'DESEMP_C_NATURALES',
       'DESEMP_SOCIALES_CIUDADANAS', 'DESEMP_INGLES', 'PUNT_GLOBAL',
       'PERCENTIL_GLOBAL', 'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL',
       'ES

In [11]:
for min_val in range(1,101):
  print(df.loc[df['PERCENTIL_GLOBAL']==min_val]['PUNT_GLOBAL'].min())

87
162
168
172
176
178
181
184
186
188
190
192
195
197
198
200
202
204
205
207
209
210
212
213
215
217
218
220
221
223
224
225
227
228
230
231
233
234
235
237
238
240
241
243
244
245
247
248
250
251
253
254
255
257
258
260
261
263
264
266
267
268
270
272
273
275
277
278
280
281
283
285
286
288
290
292
293
295
297
299
301
303
305
308
310
312
315
317
320
323
325
328
332
336
340
344
349
355
363
375


In [12]:
df.loc[df['PERCENTIL_GLOBAL']==100]['PUNT_GLOBAL'].max()

500

In [13]:
print('con decimales se tienen: ',df['ESTU_INSE_INDIVIDUAL'].nunique(),' valores únicos')
df['ESTU_INSE_INDIVIDUAL'] = df['ESTU_INSE_INDIVIDUAL'].round().astype('int')
print('con enteros se tienen: ',df['ESTU_INSE_INDIVIDUAL'].nunique(),' valores únicos')

con decimales se tienen:  366051  valores únicos
con enteros se tienen:  71  valores únicos


In [14]:
df.head(1)

,ESTU_TIPODOCUMENTO,ESTU_GENERO,PERIODO,ESTU_DEPTO_RESIDE,ESTU_MCPIO_RESIDE,ESTU_PRESENTACIONSABADO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,...,PUNT_GLOBAL,PERCENTIL_GLOBAL,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,CUARTIL_LECTURA_CRITICA,CUARTIL_MATEMATICAS,CUARTIL_C_NATURALES,CUARTIL_SOCIALES_CIUDADANAS,CUARTIL_INGLES
0,TI,F,1,CUNDINAMARCA,CHÍA,No,3,5 a 6,4.0,Educación profesional completa,...,422,100.0,68,4.0,4.0,Q4,Q4,Q4,Q4,Q4


In [15]:
df.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'FAMI_NUMLIBROS', 'FAMI_COMELECHEDERIVADOS',
       'FAMI_COMECARNEPESCADOHUEVO', 'FAMI_COMECEREALFRUTOSLEGUMBRE',
       'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA', 'COLE_NOMBRE_ESTABLECIMIENTO',
       'COLE_NATURALEZA', 'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION', 'COLE_JORNADA',
       'COLE_MCPIO_UBICACION', 'ESTU_MCPIO_PRESENTACION',
       'DESEMP_LECTURA_CRITICA', 'DESEMP_MATEMATICAS', 'DESEMP_C_NATURALES',
       'DESEMP_SOCIALES_CIUDADANAS', 'DESEMP_INGLES', 'PUNT_GLOBAL',
       'PERCENTIL_GLOBAL', 'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL',
       'ES

In [16]:
# Document type
attributes_TO_analyse = ['ESTU_DEPTO_RESIDE','ESTU_MCPIO_RESIDE','COLE_NOMBRE_ESTABLECIMIENTO','COLE_NOMBRE_SEDE','COLE_MCPIO_UBICACION',
                            'ESTU_MCPIO_PRESENTACION']
for attribute in df.columns:
  if attribute in attributes_TO_analyse:
    print(attribute,df[attribute].nunique())

ESTU_DEPTO_RESIDE 34
ESTU_MCPIO_RESIDE 1030
COLE_NOMBRE_ESTABLECIMIENTO 7020
COLE_NOMBRE_SEDE 7913
COLE_MCPIO_UBICACION 1021
ESTU_MCPIO_PRESENTACION 503


In [ ]:
attributes_TO_analyse = ['ESTU_DEPTO_RESIDE','ESTU_MCPIO_RESIDE']
for attribute in df.columns:
  if attribute in attributes_TO_analyse:
    print(attribute.capitalize())
    values_per_attribute_limited(df, attribute,0.005)

Estu_depto_reside
- EXTRANJERO: 1, porcentaje: 0.0%
Estu_mcpio_reside
- PALMAR: 15, porcentaje: 0.004%
- SAN JOAQUÍN: 11, porcentaje: 0.003%
- PANQUEBA: 18, porcentaje: 0.004%
- ENCINO: 16, porcentaje: 0.004%
- REGIDOR: 9, porcentaje: 0.002%
- YAVARATÉ: 6, porcentaje: 0.001%
- VILLA CARO: 17, porcentaje: 0.004%
- TOPAIPÍ: 12, porcentaje: 0.003%
- LA PEDRERA: 16, porcentaje: 0.004%
- GALÁN: 16, porcentaje: 0.004%
- CEPITÁ: 13, porcentaje: 0.003%
- RECETOR: 7, porcentaje: 0.002%
- PAYA: 16, porcentaje: 0.004%
- BETÉITIVA: 16, porcentaje: 0.004%
- MEDIO BAUDÓ: 17, porcentaje: 0.004%
- JORDÁN: 7, porcentaje: 0.002%
- VETAS: 13, porcentaje: 0.003%
- PISBA: 17, porcentaje: 0.004%
- MORELIA: 11, porcentaje: 0.003%
- EL ESPINO: 16, porcentaje: 0.004%
- COPER: 17, porcentaje: 0.004%
- SATIVANORTE: 17, porcentaje: 0.004%
- MAPIRIPANA: 2, porcentaje: 0.0%
- ALMEIDA: 15, porcentaje: 0.004%
- SAN JUAN DE RIOSECO: 17, porcentaje: 0.004%
- CARURÚ: 13, porcentaje: 0.003%
- SATIVASUR: 11, porcentaje: 0

In [ ]:
attributes_TO_analyse = ['COLE_MCPIO_UBICACION']
for attribute in df.columns:
  if attribute in attributes_TO_analyse:
    print(attribute.capitalize())
    values_per_attribute_limited(df, attribute,0.005)

Cole_mcpio_ubicacion
- ENCINO: 16, porcentaje: 0.004%
- REGIDOR: 10, porcentaje: 0.002%
- YAVARATÉ: 6, porcentaje: 0.001%
- VILLA CARO: 14, porcentaje: 0.003%
- TOPAIPÍ: 13, porcentaje: 0.003%
- LA PEDRERA: 17, porcentaje: 0.004%
- GUAPOTÁ: 17, porcentaje: 0.004%
- ANZÁ: 17, porcentaje: 0.004%
- CEPITÁ: 13, porcentaje: 0.003%
- RECETOR: 7, porcentaje: 0.002%
- PAYA: 16, porcentaje: 0.004%
- BETÉITIVA: 16, porcentaje: 0.004%
- MEDIO BAUDÓ: 15, porcentaje: 0.004%
- PACHAVITA: 13, porcentaje: 0.003%
- JORDÁN: 18, porcentaje: 0.004%
- VETAS: 11, porcentaje: 0.003%
- PALMAR: 15, porcentaje: 0.004%
- GACHALÁ: 15, porcentaje: 0.004%
- MORELIA: 11, porcentaje: 0.003%
- COPER: 16, porcentaje: 0.004%
- SATIVANORTE: 17, porcentaje: 0.004%
- MAPIRIPANA: 3, porcentaje: 0.001%
- ALMEIDA: 13, porcentaje: 0.003%
- SAN JUAN DE RIOSECO: 16, porcentaje: 0.004%
- ABRIAQUÍ: 18, porcentaje: 0.004%
- OICATÁ: 18, porcentaje: 0.004%
- SAN JOAQUÍN: 9, porcentaje: 0.002%
- SATIVASUR: 12, porcentaje: 0.003%
- BER

In [ ]:
attributes_TO_analyse = ['ESTU_MCPIO_PRESENTACION']
for attribute in df.columns:
  if attribute in attributes_TO_analyse:
    print(attribute.capitalize())
    values_per_attribute_limited(df, attribute, 0.005)

Estu_mcpio_presentacion
- YAVARATÉ: 6, porcentaje: 0.001%
- LA PEDRERA: 17, porcentaje: 0.004%
- MEDIO BAUDÓ: 17, porcentaje: 0.004%
- GACHALÁ: 15, porcentaje: 0.004%
- MAPIRIPANA: 3, porcentaje: 0.001%
- CARURÚ: 13, porcentaje: 0.003%
- TARAIRA: 12, porcentaje: 0.003%
- EL LITORAL DEL SAN JUAN: 13, porcentaje: 0.003%
- TARAPACÁ: 2, porcentaje: 0.0%
- GUACHENÉ: 1, porcentaje: 0.0%
- SAN FELIPE: 1, porcentaje: 0.0%
- HATO COROZAL: 1, porcentaje: 0.0%


## Filter low impact categories

In [17]:
Estu_mcpio_reside = [
    'PALMAR', 'SAN JOAQUÍN', 'PANQUEBA', 'ENCINO', 'REGIDOR', 'YAVARATÉ', 'VILLA CARO',
    'TOPAIPÍ', 'LA PEDRERA', 'GALÁN', 'CEPITÁ', 'RECETOR', 'PAYA', 'BETÉITIVA',
    'MEDIO BAUDÓ', 'JORDÁN', 'VETAS', 'PISBA', 'MORELIA', 'EL ESPINO', 'COPER',
    'SATIVANORTE', 'MAPIRIPANA', 'ALMEIDA', 'SAN JUAN DE RIOSECO', 'CARURÚ',
    'SATIVASUR', 'IZA', 'BELTRÁN', 'PAJARITO', 'SAN PEDRO DE CARTAGO', 'TARAIRA',
    'SIPÍ', 'TARAPACÁ', 'TAMBORA', 'EXTRANJERO', 'PIAMONTE', 'MEDIO ATRATO',
    'MIRITÍ - PARANÁ', 'BUSBANZÁ', 'SAN FELIPE'
]

Cole_mcpio_ubicacion = [
    'ENCINO', 'REGIDOR', 'YAVARATÉ', 'VILLA CARO', 'TOPAIPÍ', 'LA PEDRERA',
    'GUAPOTÁ', 'ANZÁ', 'CEPITÁ', 'RECETOR', 'PAYA', 'BETÉITIVA', 'MEDIO BAUDÓ',
    'PACHAVITA', 'JORDÁN', 'VETAS', 'PALMAR', 'GACHALÁ', 'MORELIA', 'COPER',
    'SATIVANORTE', 'MAPIRIPANA', 'ALMEIDA', 'SAN JUAN DE RIOSECO', 'ABRIAQUÍ',
    'OICATÁ', 'SAN JOAQUÍN', 'SATIVASUR', 'BERBEO', 'PAJARITO', 'CHARTA',
    'GALÁN', 'SAN PEDRO DE CARTAGO', 'IZA', 'SIPÍ', 'TARAPACÁ'
]

Estu_mcpio_presentacion = [
    'YAVARATÉ', 'LA PEDRERA', 'MEDIO BAUDÓ', 'GACHALÁ', 'MAPIRIPANA', 'CARURÚ',
    'TARAIRA', 'EL LITORAL DEL SAN JUAN', 'TARAPACÁ', 'GUACHENÉ', 'SAN FELIPE',
    'HATO COROZAL'
]
df_filtered = df.loc[~df['FAMI_EDUCACIONPADRE'].isin(['Ninguno','Técnica o tecnológica incompleta','Educación profesional incompleta']) &
       ~df['FAMI_EDUCACIONMADRE'].isin(['Ninguno']) &
       ~df['FAMI_TRABAJOLABORPADRE'].isin([1,8]) &
       ~df['FAMI_TRABAJOLABORMADRE'].isin([1,2,8,7]) &
       ~df['ESTU_DEPTO_RESIDE'].isin(['EXTRANJERO']) &
       ~df['ESTU_MCPIO_RESIDE'].isin(Estu_mcpio_reside) &
       ~df['COLE_MCPIO_UBICACION'].isin(Cole_mcpio_ubicacion) &
       ~df['ESTU_MCPIO_PRESENTACION'].isin(Estu_mcpio_presentacion)
       ]
df_filtered.head(1)

,ESTU_TIPODOCUMENTO,ESTU_GENERO,PERIODO,ESTU_DEPTO_RESIDE,ESTU_MCPIO_RESIDE,ESTU_PRESENTACIONSABADO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,...,PUNT_GLOBAL,PERCENTIL_GLOBAL,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,CUARTIL_LECTURA_CRITICA,CUARTIL_MATEMATICAS,CUARTIL_C_NATURALES,CUARTIL_SOCIALES_CIUDADANAS,CUARTIL_INGLES
4,TI,F,1,VALLE,CALI,No,2,1 a 2,2.0,Secundaria (Bachillerato) incompleta,...,263,39.0,51,3.0,3.0,Q2,Q2,Q1,Q2,Q3


In [18]:
df_filtered.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'FAMI_NUMLIBROS', 'FAMI_COMELECHEDERIVADOS',
       'FAMI_COMECARNEPESCADOHUEVO', 'FAMI_COMECEREALFRUTOSLEGUMBRE',
       'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA', 'COLE_NOMBRE_ESTABLECIMIENTO',
       'COLE_NATURALEZA', 'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION', 'COLE_JORNADA',
       'COLE_MCPIO_UBICACION', 'ESTU_MCPIO_PRESENTACION',
       'DESEMP_LECTURA_CRITICA', 'DESEMP_MATEMATICAS', 'DESEMP_C_NATURALES',
       'DESEMP_SOCIALES_CIUDADANAS', 'DESEMP_INGLES', 'PUNT_GLOBAL',
       'PERCENTIL_GLOBAL', 'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL',
       'ES

In [19]:
# Drops school names, high granularity and inappropiate attribute, the name doesn't affect the performance
df_filtered.drop(columns=['COLE_NOMBRE_ESTABLECIMIENTO','COLE_NOMBRE_SEDE','ESTU_MCPIO_PRESENTACION'],inplace=True)

<ipython-input-19-506b824dcc34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['COLE_NOMBRE_ESTABLECIMIENTO','COLE_NOMBRE_SEDE','ESTU_MCPIO_PRESENTACION'],inplace=True)


In [20]:
df_filtered['ALIMENTACION']=df_filtered[['FAMI_COMECARNEPESCADOHUEVO', 'FAMI_COMELECHEDERIVADOS', 'FAMI_COMECEREALFRUTOSLEGUMBRE']].astype(str).apply('-'.join, axis=1)
df_filtered['DEDICACION_ESTUDIO']=df_filtered[['ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA','FAMI_NUMLIBROS']].astype(str).apply('-'.join, axis=1)
df_filtered.drop(columns=['FAMI_COMECARNEPESCADOHUEVO', 'FAMI_COMELECHEDERIVADOS', 'FAMI_COMECEREALFRUTOSLEGUMBRE',
                          'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET','ESTU_HORASSEMANATRABAJA','FAMI_NUMLIBROS'],inplace=True)

<ipython-input-20-d0c8751e3a3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ALIMENTACION']=df_filtered[['FAMI_COMECARNEPESCADOHUEVO', 'FAMI_COMELECHEDERIVADOS', 'FAMI_COMECEREALFRUTOSLEGUMBRE']].astype(str).apply('-'.join, axis=1)
<ipython-input-20-d0c8751e3a3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DEDICACION_ESTUDIO']=df_filtered[['ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
<ipython-input-20-d0c8751e3a3c>:4: SettingWithCopyWarning: 
A value is tryin

In [21]:
df_filtered.shape

(293876, 37)

In [22]:
df_filtered.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'COLE_NATURALEZA', 'COLE_BILINGUE',
       'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'DESEMP_LECTURA_CRITICA',
       'DESEMP_MATEMATICAS', 'DESEMP_C_NATURALES',
       'DESEMP_SOCIALES_CIUDADANAS', 'DESEMP_INGLES', 'PUNT_GLOBAL',
       'PERCENTIL_GLOBAL', 'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL',
       'ESTU_NSE_ESTABLECIMIENTO', 'CUARTIL_LECTURA_CRITICA',
       'CUARTIL_MATEMATICAS', 'CUARTIL_C_NATURALES',
       'CUARTIL_SOCIALES_CIUDADANAS', 'CUARTIL_INGLES', 'ALIMENTACION',
       'DEDICACION_ESTUDIO'],
      dtype='object')

In [23]:
df_filtered['PUNT_GLOBAL'].nunique()

349

In [25]:
# Document type
attributes_NOTTO_analyse = ['ESTU_DEPTO_RESIDE','ESTU_MCPIO_RESIDE','COLE_MCPIO_UBICACION','PERCENTIL_GLOBAL','PUNT_GLOBAL','ESTU_INSE_INDIVIDUAL']
for attribute in df_filtered.columns:
  if attribute not in attributes_NOTTO_analyse:
    print(attribute.capitalize())
    values_per_attribute(df_filtered, attribute)

Estu_tipodocumento
- TI: 260806, porcentaje: 88.747%
- CC: 33070, porcentaje: 11.253%
total de categorias: 2
Estu_genero
- F: 161486, porcentaje: 54.95%
- M: 132390, porcentaje: 45.05%
total de categorias: 2
Periodo
- 1: 6448, porcentaje: 2.194%
- 2: 287428, porcentaje: 97.806%
total de categorias: 2
Estu_presentacionsabado
- No: 267712, porcentaje: 91.097%
- Si: 26164, porcentaje: 8.903%
total de categorias: 2
Fami_estratovivienda
- 2: 112347, porcentaje: 38.229%
- 1: 83871, porcentaje: 28.54%
- 3: 70432, porcentaje: 23.967%
- 4: 18381, porcentaje: 6.255%
- 6: 2969, porcentaje: 1.01%
- 5: 5876, porcentaje: 1.999%
total de categorias: 6
Fami_personashogar
- 1 a 2: 26917, porcentaje: 9.159%
- 3 a 4: 159178, porcentaje: 54.165%
- 5 a 6: 84260, porcentaje: 28.672%
- 7 a 8: 17852, porcentaje: 6.075%
- 9 o más: 5669, porcentaje: 1.929%
total de categorias: 5
Fami_cuartoshogar
- 2.0: 114452, porcentaje: 38.946%
- 3.0: 117941, porcentaje: 40.133%
- 4.0: 34442, porcentaje: 11.72%
- 6.0: 4170, 

In [ ]:
attributes_TO_analyse = ['ALIMENTACION']
for attribute in df_filtered.columns:
  if attribute in attributes_TO_analyse:
    print(attribute.capitalize())
    values_per_attribute_limited(df_filtered, attribute, 0.9)

Alimentacion
- 3-4-2: 2093, porcentaje: 0.712%
- 1-4-3: 781, porcentaje: 0.266%
- 1-4-4: 1078, porcentaje: 0.367%
- 2-2-1: 1668, porcentaje: 0.568%
- 1-4-2: 1320, porcentaje: 0.449%
- 2-1-3: 1740, porcentaje: 0.592%
- 1-1-4: 1963, porcentaje: 0.668%
- 2-4-3: 2050, porcentaje: 0.698%
- 4-1-2: 350, porcentaje: 0.119%
- 1-4-1: 652, porcentaje: 0.222%
- 3-4-1: 369, porcentaje: 0.126%
- 1-3-4: 1923, porcentaje: 0.654%
- 2-1-2: 2533, porcentaje: 0.862%
- 2-1-1: 820, porcentaje: 0.279%
- 4-2-4: 1642, porcentaje: 0.559%
- 4-2-1: 304, porcentaje: 0.103%
- 4-3-4: 420, porcentaje: 0.143%
- 3-2-1: 2630, porcentaje: 0.895%
- 4-4-3: 574, porcentaje: 0.195%
- 3-4-4: 1544, porcentaje: 0.525%
- 3-1-4: 1234, porcentaje: 0.42%
- 4-4-1: 274, porcentaje: 0.093%
- 4-3-1: 174, porcentaje: 0.059%
- 2-3-1: 1202, porcentaje: 0.409%
- 4-1-1: 193, porcentaje: 0.066%
- 2-3-4: 1975, porcentaje: 0.672%
- 4-4-2: 1887, porcentaje: 0.642%
- 4-1-3: 261, porcentaje: 0.089%
- 4-2-2: 1889, porcentaje: 0.643%
- 4-3-2: 695, 

In [ ]:
attributes_TO_analyse = ['DEDICACION_ESTUDIO']
for attribute in df_filtered.columns:
  if attribute in attributes_TO_analyse:
    print(attribute.capitalize())
    values_per_attribute_limited(df_filtered, attribute, 0.09)

Dedicacion_estudio
- 1-2-1-1: 221, porcentaje: 0.075%
- 2-2-5-2: 19, porcentaje: 0.006%
- 4-4-1-1: 118, porcentaje: 0.04%
- 3-1-2-1: 13, porcentaje: 0.004%
- 3-5-2-3: 119, porcentaje: 0.04%
- 3-1-4-1: 35, porcentaje: 0.012%
- 3-2-1-1: 169, porcentaje: 0.058%
- 2-3-3-4: 165, porcentaje: 0.056%
- 6-1-4-1: 94, porcentaje: 0.032%
- 3-4-5-3: 192, porcentaje: 0.065%
- 1-3-4-4: 113, porcentaje: 0.038%
- 2-3-3-3: 159, porcentaje: 0.054%
- 1-2-4-4: 128, porcentaje: 0.044%
- 2-2-4-1: 15, porcentaje: 0.005%
- 3-2-5-4: 47, porcentaje: 0.016%
- 3-3-4-4: 173, porcentaje: 0.059%
- 4-3-4-1: 58, porcentaje: 0.02%
- 6-3-3-1: 130, porcentaje: 0.044%
- 4-2-3-1: 120, porcentaje: 0.041%
- 6-2-5-2: 202, porcentaje: 0.069%
- 6-1-2-4: 125, porcentaje: 0.043%
- 2-1-4-3: 67, porcentaje: 0.023%
- 4-2-5-4: 112, porcentaje: 0.038%
- 3-4-2-4: 23, porcentaje: 0.008%
- 3-3-2-4: 67, porcentaje: 0.023%
- 1-3-5-3: 52, porcentaje: 0.018%
- 4-3-3-1: 150, porcentaje: 0.051%
- 4-1-2-2: 231, porcentaje: 0.079%
- 2-2-3-3: 143,

In [26]:
eliminate_values_per_attribute_limited(df_filtered, 'ALIMENTACION', 0.9)

,ESTU_TIPODOCUMENTO,ESTU_GENERO,PERIODO,ESTU_DEPTO_RESIDE,ESTU_MCPIO_RESIDE,ESTU_PRESENTACIONSABADO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,...,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,CUARTIL_LECTURA_CRITICA,CUARTIL_MATEMATICAS,CUARTIL_C_NATURALES,CUARTIL_SOCIALES_CIUDADANAS,CUARTIL_INGLES,ALIMENTACION,DEDICACION_ESTUDIO
4,TI,F,1,VALLE,CALI,No,2,1 a 2,2.0,Secundaria (Bachillerato) incompleta,...,51,3.0,3.0,Q2,Q2,Q1,Q2,Q3,1-2-2,1-1-1-3
6,TI,F,1,VALLE,CAICEDONIA,No,1,3 a 4,2.0,Primaria incompleta,...,40,1.0,3.0,Q4,Q3,Q3,Q2,Q2,4-4-4,4-1-1-3
7,TI,M,1,VALLE,JAMUNDÍ,No,1,5 a 6,2.0,Técnica o tecnológica completa,...,51,2.0,3.0,Q2,Q3,Q4,Q3,Q2,2-3-2,4-4-1-4
10,TI,F,1,VALLE,CALI,No,3,5 a 6,3.0,Primaria incompleta,...,57,3.0,3.0,Q4,Q4,Q4,Q4,Q3,3-3-3,3-1-1-4
13,TI,M,1,NARIÑO,PASTO,No,4,3 a 4,3.0,Postgrado,...,75,4.0,4.0,Q3,Q3,Q4,Q3,Q2,1-1-1,6-1-1-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551120,TI,F,2,CESAR,VALLEDUPAR,No,1,5 a 6,3.0,Educación profesional completa,...,57,3.0,3.0,Q4,Q4,Q4,Q4,Q4,2-3-2,2-2-1-2
551143,TI,F,2,TOLIMA,FRESNO,Si,1,5 a 6,2.0,Primaria incompleta,...,41,1.0,2.0,Q4,Q4,Q2,Q4,Q3,4-4-4,4-1-3-3
551144,TI,F,2,CALDAS,MARQUETALIA,Si,2,5 a 6,2.0,Primaria completa,...,43,2.0,2.0,Q1,Q1,Q1,Q2,Q1,1-2-2,1-1-1-3
551146,TI,F,2,TOLIMA,FRESNO,Si,1,5 a 6,2.0,Primaria completa,...,40,1.0,2.0,Q3,Q4,Q3,Q4,Q3,3-2-4,3-2-1-3


In [27]:
df_temp = eliminate_values_per_attribute_limited(df_filtered,'ALIMENTACION',0.9)
df_filter_low_impact = eliminate_values_per_attribute_limited(df_temp,'DEDICACION_ESTUDIO',0.09)

In [28]:
df_filter_low_impact['DEDICACION_ESTUDIO'].nunique()

167

In [29]:
df_filter_low_impact.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'COLE_NATURALEZA', 'COLE_BILINGUE',
       'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'DESEMP_LECTURA_CRITICA',
       'DESEMP_MATEMATICAS', 'DESEMP_C_NATURALES',
       'DESEMP_SOCIALES_CIUDADANAS', 'DESEMP_INGLES', 'PUNT_GLOBAL',
       'PERCENTIL_GLOBAL', 'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL',
       'ESTU_NSE_ESTABLECIMIENTO', 'CUARTIL_LECTURA_CRITICA',
       'CUARTIL_MATEMATICAS', 'CUARTIL_C_NATURALES',
       'CUARTIL_SOCIALES_CIUDADANAS', 'CUARTIL_INGLES', 'ALIMENTACION',
       'DEDICACION_ESTUDIO'],
      dtype='object')

In [ ]:
#'ESTU_DEPTO_RESIDE','ESTU_MCPIO_RESIDE','COLE_MCPIO_UBICACION'
#values_per_attribute_limited(df_filter_low_impact,'ESTU_DEPTO_RESIDE',1.5)
#values_per_attribute_limited(df_filter_low_impact,'ESTU_MCPIO_RESIDE',0.04)
#values_per_attribute_limited(df_filter_low_impact,'COLE_MCPIO_UBICACION',0.04)

In [30]:
df_temp = eliminate_values_per_attribute_limited(df_filter_low_impact,'ESTU_DEPTO_RESIDE',1.5)
values_per_attribute_limited(df_temp,'ESTU_MCPIO_RESIDE',0.04)

- VIJES: 58, porcentaje: 0.027%
- VILLA RICA: 83, porcentaje: 0.038%
- VERSALLES: 33, porcentaje: 0.015%
- SUBACHOQUE: 81, porcentaje: 0.037%
- YOTOCO: 78, porcentaje: 0.036%
- VICTORIA: 37, porcentaje: 0.017%
- TORO: 64, porcentaje: 0.03%
- MONTEBELLO: 34, porcentaje: 0.016%
- CHIPAQUE: 39, porcentaje: 0.018%
- CHALÁN: 32, porcentaje: 0.015%
- SANTA LUCÍA: 58, porcentaje: 0.027%
- CHIMÁ: 80, porcentaje: 0.037%
- SAN JACINTO DEL CAUCA: 39, porcentaje: 0.018%
- BURITICÁ: 49, porcentaje: 0.023%
- SAN CRISTÓBAL: 45, porcentaje: 0.021%
- SAN SEBASTIÁN DE BUENAVISTA: 79, porcentaje: 0.036%
- OTANCHE: 42, porcentaje: 0.019%
- SANTANA: 60, porcentaje: 0.028%
- MERCADERES: 43, porcentaje: 0.02%
- FUENTEDEORO: 72, porcentaje: 0.033%
- COLOSÓ: 49, porcentaje: 0.023%
- CAMPO DE LA CRUZ: 72, porcentaje: 0.033%
- TARQUI: 69, porcentaje: 0.032%
- SAPUYES: 32, porcentaje: 0.015%
- COVEÑAS: 71, porcentaje: 0.033%
- BUESACO: 85, porcentaje: 0.039%
- CAIMITO: 71, porcentaje: 0.033%
- LANDÁZURI: 43, porc

In [31]:
df_temp2 = eliminate_values_per_attribute_limited(df_temp,'ESTU_MCPIO_RESIDE',0.04)
values_per_attribute_limited(df_temp2,'COLE_MCPIO_UBICACION',0.04)

- SUBACHOQUE: 41, porcentaje: 0.021%
- LA APARTADA: 2, porcentaje: 0.001%
- NOBSA: 76, porcentaje: 0.039%
- VISTAHERMOSA: 2, porcentaje: 0.001%
- TONA: 1, porcentaje: 0.001%
- BARICHARA: 7, porcentaje: 0.004%
- BOYACÁ: 33, porcentaje: 0.017%
- NARIÑO: 71, porcentaje: 0.036%
- SANTO DOMINGO: 8, porcentaje: 0.004%
- MORROA: 3, porcentaje: 0.002%
- NILO: 6, porcentaje: 0.003%
- LA CELIA: 2, porcentaje: 0.001%
- ARMENIA: 4, porcentaje: 0.002%
- TOTORÓ: 2, porcentaje: 0.001%
- PASCA: 5, porcentaje: 0.003%
- FLORIÁN: 1, porcentaje: 0.001%
- PIJAO: 34, porcentaje: 0.017%
- ALTAMIRA: 3, porcentaje: 0.002%
- TAMALAMEQUE: 11, porcentaje: 0.006%
- GUALMATÁN: 2, porcentaje: 0.001%
- SUESCA: 66, porcentaje: 0.034%
- SOMONDOCO: 7, porcentaje: 0.004%
- ALDANA: 8, porcentaje: 0.004%
- COVEÑAS: 12, porcentaje: 0.006%
- EL ROBLE: 12, porcentaje: 0.006%
- NUEVA GRANADA: 10, porcentaje: 0.005%
- SUTATAUSA: 5, porcentaje: 0.003%
- ANZOÁTEGUI: 1, porcentaje: 0.001%
- NOCAIMA: 2, porcentaje: 0.001%
- CHITAGÁ

In [ ]:
values_per_attribute_limited(df_temp2,'COLE_MCPIO_UBICACION',0.07)

- SOPÓ: 130, porcentaje: 0.067%
- PATÍA: 128, porcentaje: 0.065%
- SANTA ROSA: 131, porcentaje: 0.067%
- SUBACHOQUE: 41, porcentaje: 0.021%
- CICUCO: 92, porcentaje: 0.047%
- TURBANÁ: 110, porcentaje: 0.056%
- CALAMAR: 114, porcentaje: 0.058%
- AYAPEL: 113, porcentaje: 0.058%
- SUCRE: 95, porcentaje: 0.049%
- MANATÍ: 120, porcentaje: 0.061%
- JUAN DE ACOSTA: 92, porcentaje: 0.047%
- BETULIA: 90, porcentaje: 0.046%
- MOMIL: 110, porcentaje: 0.056%
- ACHÍ: 96, porcentaje: 0.049%
- SAN BENITO ABAD: 97, porcentaje: 0.05%
- LOS PALMITOS: 110, porcentaje: 0.056%
- ANSERMA: 110, porcentaje: 0.056%
- SOPETRÁN: 94, porcentaje: 0.048%
- LURUACO: 121, porcentaje: 0.062%
- LA APARTADA: 2, porcentaje: 0.001%
- COYAIMA: 105, porcentaje: 0.054%
- SITIONUEVO: 117, porcentaje: 0.06%
- PINILLOS: 117, porcentaje: 0.06%
- LÉRIDA: 127, porcentaje: 0.065%
- CALOTO: 97, porcentaje: 0.05%
- SAN ESTANISLAO: 134, porcentaje: 0.069%
- OVEJAS: 133, porcentaje: 0.068%
- REPELÓN: 132, porcentaje: 0.068%
- PUERTO LI

In [32]:
df_filter_lowimpact = eliminate_values_per_attribute_limited(df_temp2,'COLE_MCPIO_UBICACION',0.07)
df_filter_lowimpact.shape

(180413, 37)

In [ ]:
for attribute in df_filter_lowimpact.columns:
    print(attribute.capitalize())
    values_per_attribute(df_filter_lowimpact, attribute)

Estu_tipodocumento
- TI: 162979, porcentaje: 90.337%
- CC: 17434, porcentaje: 9.663%
total de categorias: 2
Estu_genero
- F: 99512, porcentaje: 55.158%
- M: 80901, porcentaje: 44.842%
total de categorias: 2
Periodo
- 1: 5499, porcentaje: 3.048%
- 2: 174914, porcentaje: 96.952%
total de categorias: 2
Estu_depto_reside
- VALLE: 17953, porcentaje: 9.951%
- NARIÑO: 4000, porcentaje: 2.217%
- ATLANTICO: 12562, porcentaje: 6.963%
- BOGOTÁ: 34654, porcentaje: 19.208%
- CUNDINAMARCA: 10608, porcentaje: 5.88%
- ANTIOQUIA: 26018, porcentaje: 14.421%
- CAUCA: 3149, porcentaje: 1.745%
- HUILA: 3935, porcentaje: 2.181%
- SANTANDER: 9768, porcentaje: 5.414%
- RISARALDA: 4190, porcentaje: 2.322%
- BOLIVAR: 8488, porcentaje: 4.705%
- CESAR: 5094, porcentaje: 2.824%
- CORDOBA: 7615, porcentaje: 4.221%
- CALDAS: 3133, porcentaje: 1.737%
- MAGDALENA: 5176, porcentaje: 2.869%
- NORTE SANTANDER: 6815, porcentaje: 3.777%
- SUCRE: 3776, porcentaje: 2.093%
- META: 3782, porcentaje: 2.096%
- TOLIMA: 5440, porc

In [33]:
df_filter_lowimpact['PUNT_GLOBAL'].nunique()

347

In [ ]:
df_filter_lowimpact.shape

(180413, 36)

In [ ]:
df_filter_lowimpact.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'COLE_NATURALEZA', 'COLE_BILINGUE',
       'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'DESEMP_LECTURA_CRITICA',
       'DESEMP_MATEMATICAS', 'DESEMP_C_NATURALES',
       'DESEMP_SOCIALES_CIUDADANAS', 'DESEMP_INGLES', 'PERCENTIL_GLOBAL',
       'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL',
       'ESTU_NSE_ESTABLECIMIENTO', 'CUARTIL_LECTURA_CRITICA',
       'CUARTIL_MATEMATICAS', 'CUARTIL_C_NATURALES',
       'CUARTIL_SOCIALES_CIUDADANAS', 'CUARTIL_INGLES', 'ALIMENTACION',
       'DEDICACION_ESTUDIO'],
      dtype='object')

In [ ]:
df_filter_lowimpact.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180413 entries, 4 to 551147
Data columns (total 36 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ESTU_TIPODOCUMENTO           180413 non-null  object 
 1   ESTU_GENERO                  180413 non-null  object 
 2   PERIODO                      180413 non-null  int64  
 3   ESTU_DEPTO_RESIDE            180413 non-null  object 
 4   ESTU_MCPIO_RESIDE            180413 non-null  object 
 5   ESTU_PRESENTACIONSABADO      180413 non-null  object 
 6   FAMI_ESTRATOVIVIENDA         180413 non-null  int64  
 7   FAMI_PERSONASHOGAR           180413 non-null  object 
 8   FAMI_CUARTOSHOGAR            180413 non-null  float64
 9   FAMI_EDUCACIONPADRE          180413 non-null  object 
 10  FAMI_EDUCACIONMADRE          180413 non-null  object 
 11  FAMI_TRABAJOLABORPADRE       180413 non-null  int64  
 12  FAMI_TRABAJOLABORMADRE       180413 non-null  int64  
 13  COLE

## Export dataframe to file

In [34]:
df_filter_lowimpact.to_parquet('/content/drive/MyDrive/No Country Proyecto/Data/Clean/Icfes2023_feature_engineering.parquet')

In [35]:
# Prueba
df_prueba = pd.read_parquet('/content/drive/MyDrive/No Country Proyecto/Data/Clean/Icfes2023_feature_engineering.parquet')
df_prueba.head(1)

,ESTU_TIPODOCUMENTO,ESTU_GENERO,PERIODO,ESTU_DEPTO_RESIDE,ESTU_MCPIO_RESIDE,ESTU_PRESENTACIONSABADO,FAMI_ESTRATOVIVIENDA,FAMI_PERSONASHOGAR,FAMI_CUARTOSHOGAR,FAMI_EDUCACIONPADRE,...,ESTU_INSE_INDIVIDUAL,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,CUARTIL_LECTURA_CRITICA,CUARTIL_MATEMATICAS,CUARTIL_C_NATURALES,CUARTIL_SOCIALES_CIUDADANAS,CUARTIL_INGLES,ALIMENTACION,DEDICACION_ESTUDIO
4,TI,F,1,VALLE,CALI,No,2,1 a 2,2.0,Secundaria (Bachillerato) incompleta,...,51,3.0,3.0,Q2,Q2,Q1,Q2,Q3,1-2-2,1-1-1-3


In [36]:
df_prueba.shape

(180413, 37)

In [37]:
df_prueba.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_GENERO', 'PERIODO', 'ESTU_DEPTO_RESIDE',
       'ESTU_MCPIO_RESIDE', 'ESTU_PRESENTACIONSABADO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'COLE_NATURALEZA', 'COLE_BILINGUE',
       'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL', 'COLE_AREA_UBICACION',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'DESEMP_LECTURA_CRITICA',
       'DESEMP_MATEMATICAS', 'DESEMP_C_NATURALES',
       'DESEMP_SOCIALES_CIUDADANAS', 'DESEMP_INGLES', 'PUNT_GLOBAL',
       'PERCENTIL_GLOBAL', 'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL',
       'ESTU_NSE_ESTABLECIMIENTO', 'CUARTIL_LECTURA_CRITICA',
       'CUARTIL_MATEMATICAS', 'CUARTIL_C_NATURALES',
       'CUARTIL_SOCIALES_CIUDADANAS', 'CUARTIL_INGLES', 'ALIMENTACION',
       'DEDICACION_ESTUDIO'],
      dtype='object')